In [1]:
import geopandas as gpd
from geopandas.tools import sjoin
import pandas as pd
import requests
import json
from pprint import pprint
from shapely.geometry import Point
import matplotlib.pyplot as plt
import numpy as np
from itertools import compress
import os


from rdb_functions import generate_b1_b0
from rdb_functions import fill_nas
pd.set_option('display.max_columns', None)

### Change the next values to import and change the data

You can download the mexican census data in the next links: 

<ol>
  <li><a href="https://www.inegi.org.mx/programas/ccpv/2010/#Datos_abiertos">Census Data 2010</a></li>
  <li><a href="https://www.inegi.org.mx/programas/ccpv/2020/#Datos_abiertos">Census Data 2020</a></li>
</ol>

In [75]:
#Paarameters

zm_n=21
muns='14'
year=2020

In [76]:
zm=pd.read_excel(f"/Volumes/TOSHIBA EXT/BootCamp/ZMs/ZM_{zm_n}.xlsx")

data=pd.read_csv(f"/Volumes/TOSHIBA EXT/BootCamp/Datos/inegi/ageb_mza_urbana_{muns}_cpv{year}/conjunto_de_datos/conjunto_de_datos_ageb_urbana_{muns}_cpv{year}.csv")
#data=pd.read_csv(f"/Volumes/TOSHIBA EXT/BootCamp/Datos/inegi/resultados_ageb_urbana_{muns}_cpv2010/conjunto_de_datos/resultados_ageb_urbana_{muns}_cpv2010.csv")

data.columns= data.columns.str.lower()

data['clave']=data['entidad']*1000+data['mun']

cl=[i in set(zm.clave) for i in data['clave']]
data=data[cl]

### Removing aggregated data and special characters

In [77]:
data.replace('^\*+$', np.nan, regex=True,inplace=True)
data.replace('N/D', np.nan, regex=True,inplace=True)
data=data[data['mza']!=0].reset_index(drop=True)

### All missing values were replaced using a linear regression with total population as predictor

In [78]:
variables_kept=['clave','loc','ageb','mza','pobtot','pocupada_f', 'pocupada', 'vph_autom', 'vivtot', 'vph_pc']

data=data[variables_kept]

variables_num=['pobtot','pocupada_f', 'pocupada', 'vph_autom', 'vivtot', 'vph_pc']

for c in variables_num:
    data[c]=pd.to_numeric(data[c])
    
for column in list(variables_num):
    b1_b0=generate_b1_b0(data,'pobtot',column)
    data[column]=fill_nas(data[column],data['pobtot'],b1_b0)

def zero_plus(item,digits):
    i=str(item)
    zeros=digits-len(i)
    return zeros*'0'+i

data['loc']=[zero_plus(i,4) for i in data['loc']]
data['ageb']=[zero_plus(i,4) for i in data['ageb']]
data['mza']=[zero_plus(i,3) for i in data['mza']]

data['id']=data['clave'].astype(str)+data['loc']+data['ageb']

data=data.groupby(['clave','loc','ageb','id'],as_index=False).sum()

data['pct_wf']  = data['pocupada_f']/data['pocupada']*100

data['pct_hh_car'] = data['vph_autom']/data['vivtot']*100

data['pct_hh_pc'] = data['vph_pc']/data['vivtot']*100

path=f'/Volumes/TOSHIBA EXT/BootCamp/Datos/inegi/sociodem_{zm_n}_{muns}_{year}.csv'
data.to_csv(path,index=False)